# Data Analysis for Kentucky Natural Disasters Story

## Import Libraries

In [1]:
import pandas as pd
import requests

## Read data from FEMA website

In [2]:
df_pa = pd.read_csv("https://www.fema.gov/api/open/v1/PublicAssistanceFundedProjectsSummaries.csv")
print(df_pa.shape)
df_pa.head()

KeyboardInterrupt: 

In [ ]:
df_hm = pd.read_csv("https://www.fema.gov/api/open/v1/HazardMitigationAssistanceProjects.csv")
print(df_hm.shape)
df_hm.head()

In [ ]:
df_decs = pd.read_csv("https://www.fema.gov/api/open/v1/FemaWebDisasterDeclarations.csv")
print(df_decs.shape)
df_decs.head()

In [ ]:
petition_to_challenge_states = ['Arizona','Utah','Colorado','Wyoming','Montana','North Dakota','South Dakota','Nebraska','Kansas','Oklahoma','Texas','Louisiana','Arkansas','Missouri','Wisconsin','Indiana','Michigan','Ohio','Mississippi','Alabama','Georgia','Florida','South Carolina','New Jersey','West Virginia','Kentucky']
print(len(petition_to_challenge_states))
oppose_as_amicus_states = ['Nevada']
print(len(oppose_as_amicus_states))
all_oppose_states = petition_to_challenge_states + oppose_as_amicus_states
supporting_as_intervenors_states = ['Washington','Oregon','California','Hawaii','District of Columbia','Maine','New Mexico','Minnesota','Iowa','Illinois','New York','Virginia','Delaware','Vermont','New Hampshire','Rhode Island','Maryland','Massachusetts','Connecticut'] 
print(len(supporting_as_intervenors_states))
not_party_to_litigation_states = ['Alaska','Idaho','Tennessee','North Carolina','Pennsylvania']
print(len(supporting_as_intervenors_states))
all_states_and_DC = all_oppose_states + supporting_as_intervenors_states + not_party_to_litigation_states
print(len(set(all_states_and_DC)))

## Filtering

### By year

In [ ]:
df_decs_decade = df_decs.loc[(df_decs['incidentBeginDate']>="2009-01-01T00:00:00.000Z")&
                         (df_decs['incidentBeginDate']<"2019-01-01T00:00:00.000Z"),'disasterNumber'].copy()

In [ ]:
df_decs_decade.min(),df_decs_decade.max()

In [ ]:
df_hm_decade = df_hm[df_hm['disasterNumber'].isin(df_decs_decade)]
print(df_hm_decade.shape)

In [ ]:
df_hm_decade['disasterNumber'].min(),df_hm_decade['disasterNumber'].max()

In [ ]:
df_pa_decade = df_pa[df_pa['disasterNumber'].isin(df_decs_decade)]
print(df_pa_decade.shape)

In [ ]:
df_pa_decade['disasterNumber'].min(),df_pa_decade['disasterNumber'].max()

### By sub-program of Haz Mit

In [ ]:
df_hmgp_decade = df_hm_decade[df_hm_decade['programArea']=='HMGP'].copy()
print(df_hmgp_decade.shape)

### By state-clean-power-plan status

In [ ]:
df_pa_decade_all_oppose_states = df_pa_decade[df_pa_decade['state'].isin(all_oppose_states)].copy()
print(df_pa_decade_all_oppose_states.shape)

df_hmgp_decade_all_oppose_states = df_hmgp_decade[df_hmgp_decade['state'].isin(all_oppose_states)].copy()
print(df_hmgp_decade_all_oppose_states.shape)

In [ ]:
df_pa_decade_all_oppose_states['state'].nunique(),df_hmgp_decade_all_oppose_states['state'].nunique()

In [ ]:
# Total federal obligated amount across all public assistance projects for declared disasters which began...
# ... between January 1st, 2009 and December 31st, 2018 in states that either petitioned to challenge the clean power plan...
# ... or opposed it as amicus curiae.
df_pa_decade_all_oppose_states['federalObligatedAmount'].sum()

In [ ]:
# Total federal share obligated amount across all hazard mitigation grant program projects for declared disasters which began...
# ... between January 1st, 2009 and December 31st, 2018 in states that either petitioned to challenge the clean power plan...
# ... or opposed it as amicus curiae.
df_hmgp_decade_all_oppose_states['federalShareObligated'].sum()

## Compare states

In [ ]:
print(df_pa_decade['federalObligatedAmount'].sum())
print(df_pa_decade_all_oppose_states['federalObligatedAmount'].sum() / df_pa_decade['federalObligatedAmount'].sum())

In [ ]:
print(df_hmgp_decade['federalShareObligated'].sum())
print(df_hmgp_decade_all_oppose_states['federalShareObligated'].sum() / df_hmgp_decade['federalShareObligated'].sum())

In [ ]:
df_funding_by_state = pd.DataFrame(data = {'State': all_states_and_DC})
df_funding_by_state.sort_values(by = 'State', inplace = True)
df_funding_by_state.reset_index(inplace = True, drop = True)
df_funding_by_state['Clean Power Plan'] = df_funding_by_state['State'].apply(lambda x: "Oppose" if x in all_oppose_states else "Not Oppose")
df_funding_by_state['Public Assistance'] = df_funding_by_state['State'].apply(lambda x:
                df_pa_decade.loc[df_pa_decade['state']==x,'federalObligatedAmount'].sum())
df_funding_by_state['HMGP'] = df_funding_by_state['State'].apply(lambda x:
                df_hmgp_decade.loc[df_hmgp_decade['state']==x,'federalShareObligated'].sum())
df_funding_by_state['Total'] = df_funding_by_state['Public Assistance'] + df_funding_by_state['HMGP']
df_funding_by_state

In [ ]:
#df_funding_by_state.sort_values(by = "Public Assistance", ascending = False).head(10).to_csv("top_public_assistance_funding", index = False)
df_funding_by_state.sort_values(by = "Public Assistance", ascending = False).head(10)

In [ ]:
#df_funding_by_state.sort_values(by = "HMGP", ascending = False).head(10).to_csv("top_hmgp_funding", index = False)
df_funding_by_state.sort_values(by = "HMGP", ascending = False).head(10)

In [ ]:
#df_funding_by_state.sort_values(by = "Total", ascending = False).head(10).to_csv("top_total_funding", index = False)
df_funding_by_state.sort_values(by = "Total", ascending = False).head(10)

Add GHG and GHG per capita (for now, just CO2)

In [ ]:
# https://www.eia.gov/environment/emissions/state/
url = 'https://www.eia.gov/environment/emissions/state/excel/summary_2016.xlsx'
df_co2_emissions = pd.read_excel(url, skiprows = 2, index_col = 0)
print(df_co2_emissions.shape)
df_co2_emissions.head()

In [ ]:
key = open('census_api_key.txt','r').read()

In [ ]:
population = "B01001_001E"
base_url = "https://api.census.gov/data/2016/acs/acs5?"
end_url = "get=NAME,%s&for=state:*&key=%s" % (population, key)

response = requests.get(base_url+end_url)
print(response)
response_json = response.json()
df_population = pd.DataFrame(response_json[1:],
                                columns = response_json[0])
df_population.set_index("NAME", inplace = True)
df_population.head()

In [ ]:
# df_funding_by_state['CO2 2007-16'] = df_funding_by_state['State'].apply(lambda x:
#             df_co2_emissions.loc[x,2007:2017].sum())
df_funding_by_state['CO2 2016 (Millions of Metric Tons)'] = df_funding_by_state['State'].apply(lambda x:
            df_co2_emissions.loc[x,2016])
df_funding_by_state['CO2 2016 (Metric Tons)'] = df_funding_by_state['CO2 2016 (Millions of Metric Tons)']*(10**6)
df_funding_by_state['Pop 2016'] = df_funding_by_state['State'].apply(lambda x:
        df_population.loc[x,population]).apply(int)
df_funding_by_state['CO2 Emissions (Metric Tons Per Capita) 2016'] = df_funding_by_state['CO2 2016 (Metric Tons)'] / df_funding_by_state['Pop 2016']
df_funding_by_state.head()

In [ ]:
#df_funding_by_state.to_csv("fema_funding_by_state", index = False)